In [1]:
import numpy as np
import pandas as pd
from doubleml.data.panel_data import DoubleMLPanelData
from doubleml.plm.plpr import DoubleMLPLPR
from sklearn.linear_model import LassoCV
from sklearn.base import clone
from sklearn.tree import DecisionTreeRegressor
from lightgbm import LGBMRegressor
# from doubleml.plm.utils._plpr_util import extend_data, cre_fct, fd_fct, wd_fct
from doubleml.plm.datasets.dgp_static_panel_CP2025 import make_static_panel_CP2025
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
import warnings
warnings.filterwarnings("ignore")

In [3]:
class PolyPlus(BaseEstimator, TransformerMixin):
    """PolynomialFeatures(degree=k) and additional terms x_i^(k+1)."""

    def __init__(self, degree=2, interaction_only=False, include_bias=False):
        self.degree = degree
        self.extra_degree = degree + 1
        self.interaction_only = interaction_only
        self.include_bias = include_bias
        self.poly = PolynomialFeatures(degree=degree, interaction_only=interaction_only, include_bias=include_bias)

    def fit(self, X, y=None):
        self.poly.fit(X)
        self.n_features_in_ = X.shape[1]
        return self

    def transform(self, X):
        X = np.asarray(X)
        X_poly = self.poly.transform(X)
        X_extra = X ** self.extra_degree
        return np.hstack([X_poly, X_extra])

    def get_feature_names_out(self, input_features=None):
        input_features = np.array(
            input_features
            if input_features is not None
            else [f"x{i}" for i in range(self.n_features_in_)]
        )
        poly_names = self.poly.get_feature_names_out(input_features)
        extra_names = [f"{name}^{self.extra_degree}" for name in input_features]
        return np.concatenate([poly_names, extra_names])

In [4]:
dim_x = 30
indices_x = [x for x in range(dim_x)]
indices_x_tr = [x + dim_x for x in indices_x]

preprocessor = ColumnTransformer([
    ('poly_x', make_pipeline(
        PolyPlus(degree=2, include_bias=False, interaction_only=False)
    ), indices_x),       
    ('poly_x_mean', make_pipeline(
        PolyPlus(degree=2, include_bias=False, interaction_only=False)
    ), indices_x_tr) 
], remainder='passthrough')

preprocessor_wg = ColumnTransformer([
    ('poly_x', make_pipeline(
        PolyPlus(degree=2, include_bias=False, interaction_only=False)
    ), indices_x),       
], remainder='passthrough')

ml_lasso = make_pipeline(
    preprocessor,
    StandardScaler(),
    LassoCV(n_alphas=20, cv=2, n_jobs=5)
)

ml_lasso_wg = make_pipeline(
    preprocessor_wg,
    StandardScaler(),
    LassoCV(n_alphas=20, cv=2, n_jobs=5)
)

ml_cart = DecisionTreeRegressor()

ml_boost = LGBMRegressor(verbose=-1, 
                         n_estimators=100, 
                         learning_rate=0.3,
                         min_child_samples=1) 

In [5]:
data = make_static_panel_CP2025(num_n=100, dgp_type='dgp1', x_var=5, a_var=0.95)
data

,id,time,y,d,x1,x2,x3,x4,x5,x6,...,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30
0,1,1,-1.767568,-1.348237,0.492677,0.281933,-2.142618,0.791443,2.976620,-1.001715,...,-1.198500,-0.049211,0.600889,2.435667,-1.387149,3.034459,-0.062419,0.258166,-1.168477,-1.061057
1,1,2,-5.095199,-3.566642,-1.608388,-0.819905,-3.570497,1.583374,1.644214,-4.221177,...,4.517936,0.413499,2.150563,-2.971910,0.922270,-2.628696,-1.772420,-3.851087,3.270008,0.820763
2,1,3,6.437985,4.202518,-1.050850,-1.400580,3.183289,3.513685,1.861339,0.888485,...,1.163815,-0.069711,-0.202117,-1.262765,1.133570,0.884130,0.484024,3.124910,0.004369,0.349072
3,1,4,1.692969,0.776318,-0.275264,-0.787588,-1.492324,-3.920095,-2.246768,-1.655923,...,-1.120645,-1.726098,-2.561617,-2.247641,0.685799,3.943749,2.891479,5.381948,1.455669,-2.480590
4,1,5,-0.760349,-0.423762,-5.334104,-3.650439,0.447345,-4.053885,1.367370,0.752763,...,-1.666836,-0.607192,3.658921,-1.153617,1.338251,2.990290,-0.717240,2.494413,-0.576748,-0.049214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,100,6,6.736806,3.652808,0.935941,-1.046372,2.972970,-3.188139,-1.368655,-1.973138,...,-0.693739,1.142071,-1.017755,4.427896,0.988486,-0.082238,1.047705,1.222772,3.264437,1.173541
996,100,7,6.023375,3.868859,-0.667464,5.690558,4.230361,0.512239,-0.016779,-1.998309,...,0.429095,-0.879970,-0.333486,0.856337,3.159868,-2.772002,1.782697,1.158639,1.373919,-3.298531
997,100,8,3.110866,0.955741,1.789798,-1.256050,-0.369970,-1.965363,0.805850,-0.837638,...,0.314910,2.215582,1.695434,-0.092416,-0.691905,2.864254,1.673074,-0.839339,1.389325,1.714632
998,100,9,4.572219,3.239447,-3.129920,-1.654972,3.222430,-1.193018,0.287887,-0.382851,...,-0.481192,1.541299,1.153674,-4.382081,4.017794,1.117018,-1.648193,-1.139779,2.748231,3.032575


In [ ]:
def run_sim(n_reps, num_n, dim_x=30, theta=0.5, dgp_type='dgp3'):

    approaches = ["cre_general", "cre_normal", "fd_exact", "wg_approx"]
    models  = ["lasso", "cart", "boost"]

    res = {
        d: {
            m: np.full((n_reps, 3), np.nan)
            for m in models
        }
        for d in approaches
    }

    x_cols = [f"x{i+1}" for i in range(dim_x)]

    def run_single_dml(dml_data, ml, approach, grid=None):
        est = DoubleMLPLPR(dml_data, clone(ml), clone(ml), approach=approach, n_folds=5)

        if grid is not None:
            est.tune(param_grids=grid, search_mode='randomized_search',
                     n_iter_randomized_search=5, n_jobs_cv=5)

        est.fit()

        coef_err = est.coef[0] - theta
        se       = est.se[0]
        conf     = est.confint()
        covered  = (conf['2.5 %'].iloc[0] <= theta) & (conf['97.5 %'].iloc[0] >= theta)

        return coef_err, se, covered

    for i in range(n_reps):

        print(f"\rProcessing: {round((i+1)/n_reps*100, 3)} %", end="")

        cart_grid = {'ml_l': {'ccp_alpha': np.random.choice(np.arange(0.002, 0.052 , 0.002), 5, replace=False),
                              'max_depth': np.random.choice(np.arange(2, 11 , 1), 5, replace=False)},
                     'ml_m': {'ccp_alpha': np.random.choice(np.arange(0.002, 0.052 , 0.002), 5, replace=False),
                              'max_depth': np.random.choice(np.arange(2, 11 , 1), 5, replace=False)}}

        boost_grid= {'ml_l': {'reg_lambda': np.random.choice(np.arange(0.2, 2 , 0.2), 5, replace=False), 
                              'max_depth': np.random.choice(np.arange(2, 11 , 1), 5, replace=False)},
                     'ml_m': {'reg_lambda': np.random.choice(np.arange(0.2, 2 , 0.2), 5, replace=False),
                              'max_depth': np.random.choice(np.arange(2, 11 , 1), 5, replace=False)}}

        data = make_static_panel_CP2025(num_n=num_n, dim_x=dim_x, theta=theta, dgp_type=dgp_type, x_var=5, a_var=0.95)
        dml_data = DoubleMLPanelData(data, y_col='y', d_cols='d', t_col='time', id_col='id', x_cols=x_cols, static_panel=True)

        # CRE general
        res['cre_general']['lasso'][i, :] = run_single_dml(dml_data, ml_lasso, 'cre_general', grid=None)
        res['cre_general']['cart'][i, :] = run_single_dml(dml_data, ml_cart, 'cre_general', grid=cart_grid)
        res['cre_general']['boost'][i, :] = run_single_dml(dml_data, ml_boost, 'cre_general', grid=boost_grid)

        # CRE normal
        res['cre_normal']['lasso'][i, :] = run_single_dml(dml_data, ml_lasso, 'cre_normal', grid=None)
        res['cre_normal']['cart'][i, :] = run_single_dml(dml_data, ml_cart, 'cre_normal', grid=cart_grid)
        res['cre_normal']['boost'][i, :] = run_single_dml(dml_data, ml_boost, 'cre_normal', grid=boost_grid)

        # FD
        res['fd_exact']['lasso'][i, :] = run_single_dml(dml_data, ml_lasso, 'fd_exact', grid=None)
        res['fd_exact']['cart'][i, :] = run_single_dml(dml_data, ml_cart, 'fd_exact', grid=cart_grid)
        res['fd_exact']['boost'][i, :] = run_single_dml(dml_data, ml_boost, 'fd_exact', grid=boost_grid)

        # WD
        res['wg_approx']['lasso'][i, :] = run_single_dml(dml_data, ml_lasso_wg, 'wg_approx', grid=None)
        res['wg_approx']['cart'][i, :] = run_single_dml(dml_data, ml_cart, 'wg_approx', grid=cart_grid)
        res['wg_approx']['boost'][i, :] = run_single_dml(dml_data, ml_boost, 'wg_approx', grid=boost_grid)

    # summary
    rows = []
    index = []

    for approach, models_dict in res.items():
        for model, arr in models_dict.items():

            bias = np.mean(arr[:, 0])
            se_mean = np.mean(arr[:, 1])
            sd = np.std(arr[:, 1])
            coverage = np.mean(arr[:, 2])
            se_over_sd = sd / se_mean if se_mean > 0 else np.nan
            rmse = np.sqrt(np.mean(arr[:, 0]**2))

            rows.append([bias, rmse, se_over_sd, coverage])
            index.append((approach, model))      

    summary = pd.DataFrame(
        rows,
        index=pd.MultiIndex.from_tuples(index, names=["Approach", "ML Model"]),
        columns=["Bias", "RMSE", "SE/SD", "Coverage"]
    )

    return summary

In [14]:
np.random.seed(123)

res_dgp1 = run_sim(n_reps=100, num_n=100, theta=0.5, dgp_type='dgp1')
res_dgp1

Processing: 100.0 %

Bias      RMSE     SE/SD  Coverage
Approach    ML Model                                        
cre_general lasso     0.024030  0.040756  0.076669      0.92
            cart     -0.033091  0.069709  0.112711      0.75
            boost    -0.041728  0.062508  0.114469      0.76
cre_normal  lasso     0.094727  0.103281  0.111451      0.41
            cart     -0.015669  0.072878  0.125903      0.89
            boost     0.051158  0.081405  0.128427      0.90
fd_exact    lasso     0.021691  0.044563  0.084716      0.90
            cart      0.076125  0.105962  0.099474      0.62
            boost     0.010003  0.049025  0.116624      0.87
wg_approx   lasso     0.003884  0.032382  0.077256      0.97
            cart     -0.002517  0.048548  0.090029      0.87
            boost    -0.042819  0.058042  0.086614      0.71

In [16]:
np.random.seed(123)

res_dgp2 = run_sim(n_reps=100, num_n=100, theta=0.5, dgp_type='dgp2')
res_dgp2

Processing: 100.0 %

Bias      RMSE     SE/SD  Coverage
Approach    ML Model                                        
cre_general lasso    -0.005823  0.027244  0.073960      0.95
            cart     -0.109613  0.122162  0.106904      0.24
            boost    -0.063039  0.075676  0.081278      0.50
cre_normal  lasso     0.070362  0.076794  0.087415      0.40
            cart     -0.028230  0.073113  0.182335      0.94
            boost    -0.016764  0.060976  0.183879      0.89
fd_exact    lasso    -0.002715  0.032797  0.091710      0.95
            cart     -0.074209  0.083379  0.099775      0.47
            boost    -0.062527  0.076963  0.085480      0.58
wg_approx   lasso    -0.003015  0.027330  0.074295      0.95
            cart     -0.024405  0.038327  0.079555      0.90
            boost    -0.056258  0.067245  0.083798      0.52

In [17]:
np.random.seed(123)

run_sim(n_reps=100, num_n=100, theta=0.5, dgp_type='dgp3')

Processing: 100.0 %

Bias      RMSE     SE/SD  Coverage
Approach    ML Model                                        
cre_general lasso     0.016582  0.037272  0.080832      0.94
            cart      0.493280  0.534554  0.275800      0.05
            boost     0.443519  0.451916  0.180075      0.00
cre_normal  lasso     0.083721  0.100538  0.129024      0.71
            cart      0.413418  0.468509  0.218055      0.13
            boost     0.385419  0.393097  0.185301      0.00
fd_exact    lasso     0.014759  0.041217  0.086594      0.92
            cart      0.658754  0.673590  0.259508      0.00
            boost     0.588552  0.593496  0.134829      0.00
wg_approx   lasso     0.630003  0.630942  0.099154      0.00
            cart      0.515894  0.532497  0.227144      0.01
            boost     0.593587  0.597220  0.137237      0.00